In [3]:
import praw
import random
import json
import pickle

In [10]:
reddit = praw.Reddit(client_id='i9OxQyEl0ikmDQ', client_secret='hg0viho18o2lUIbMT3bSpYhN1KGXTA', user_agent='Scraping')

In [5]:
topics = ["animal", "news", "food", "travel", "sport", "science", "music", "game", "book", "movie"]

In [5]:
#titles

def parse_submission(submission_post):
    submission_dict = {}
    submission_dict["submission"] = {"author": getattr(submission_post.author, 'name', None), 
                                     "text": submission_post.selftext,
                                     "subreddit": submission_post.subreddit.display_name,
                                     "title": submission_post.title}
    submission_dict["comments"] = []
    for comment in submission_post.comments:
        if len(submission_dict["comments"]) < 20:
            submission_dict["comments"].append({"author": getattr(comment.author, 'name', None), "text": comment.body})
    return submission_dict

In [13]:
#titles

def parse_submission(submission_post):
    submission_dict = {}
    submission_dict["submission"] = {"author": getattr(submission_post.author, 'name', None), 
                                     "text": submission_post.selftext,
                                     "title": submission_post.title}
    submission_dict["comments"] = []
    return submission_dict

In [14]:
%%time

user2subreddits = {}
parsed_submissions = {}
subreddit2submission = {}

current_users = []

count = 0

for topic in topics:
    hot_posts = reddit.subreddit(topic).hot(limit=2)
    for post in hot_posts:
        topic_user = post.author.name
    current_users.append(topic_user)
    
for i in range(1):
    following_users = []
    print(count)
    count += 1
    
    for current_user in current_users:
        
        current_subreddits = []
        
        submission_posts = reddit.redditor(current_user).submissions.hot(limit=10)
        comment_posts = reddit.redditor(current_user).comments.hot(limit=10)
        
        user2subreddits[current_user] = {"submissions": [], "comments": []}
                
        for post in submission_posts: 
            current_subreddit = post.subreddit.display_name
            current_subreddits.append(current_subreddit)
            user2subreddits[current_user]["submissions"].append({post.id: current_subreddit})
                                    
        for post in comment_posts:
            current_subreddit = post.subreddit.display_name
            current_subreddits.append(current_subreddit)
            user2subreddits[current_user]["comments"].append({post.id: current_subreddit})
                        
        for subred in current_subreddits:
            subreddit2submission[subred] = []
            subreddit_posts = reddit.subreddit(subred).hot(limit=10)
            for post in subreddit_posts:
                subreddit2submission[subred].append(post.id)
                parsed_post = parse_submission(post)
                if parsed_post["submission"]["author"] not in following_users:
                    following_users.append(parsed_post["submission"]["author"])
                parsed_submissions[post.id] = parsed_post
                
    current_users = following_users

0
SUBMISSION
COMMENTS
SUBREDDITS
SUBMISSION
COMMENTS
SUBREDDITS
SUBMISSION
COMMENTS
SUBREDDITS
SUBMISSION
COMMENTS
SUBREDDITS
SUBMISSION
COMMENTS
SUBREDDITS
SUBMISSION
COMMENTS
SUBREDDITS
SUBMISSION
COMMENTS
SUBREDDITS
SUBMISSION
COMMENTS
SUBREDDITS
SUBMISSION
COMMENTS
SUBREDDITS
SUBMISSION
COMMENTS
SUBREDDITS
CPU times: user 2.45 s, sys: 184 ms, total: 2.63 s
Wall time: 7min 2s


In [ ]:
%%time

user2subreddits = {}
parsed_submissions = {}
subreddit2submission = {}

current_users = []

count = 0

for topic in topics:
    hot_posts = reddit.subreddit(topic).hot(limit=2)
    for post in hot_posts:
        topic_submission = post.title
        topic_user = post.author.name
    current_users.append(topic_user)
    
for i in range(1):
    following_users = []
    for current_user in current_users:
        current_subreddits = []
        submission_posts = reddit.redditor(current_user).submissions.hot(limit=10)
        comment_posts = reddit.redditor(current_user).comments.hot(limit=10)
        user2subreddits[current_user] = {"submissions": [], "comments": []}
        for post in submission_posts: 
            user2subreddits[current_user]["submissions"].append(post.id)
            if post.id not in list(parsed_submissions.keys()):
                parsed_post = parse_submission(post)
                if parsed_post["submission"]["subreddit"] not in current_subreddits:
                    current_subreddits.append(parsed_post["submission"]["subreddit"])
                for u in parsed_post["comments"]:
                    if u["author"] and u["author"] not in following_users:
                        following_users.append(u["author"])
                parsed_submissions[post.id] = parsed_post
        
        for comm in comment_posts:
            user2subreddits[current_user]["comments"].append(comm.submission.id)
            if comm.submission.id not in list(parsed_submissions.keys()):
                parsed_post = parse_submission(comm.submission)
                if parsed_post["submission"]["subreddit"] not in current_subreddits:
                    current_subreddits.append(parsed_post["submission"]["subreddit"])
                for u in parsed_post["comments"]:
                    if u["author"] and u["author"] != current_user and u["author"] not in following_users:
                        following_users.append(u["author"])
                if parsed_post["submission"]["author"] not in following_users:
                    following_users.append(parsed_post["submission"]["author"])
                parsed_submissions[comm.submission.id] = parsed_post
            
        for subred in current_subreddits:
            subreddit2submission[subred] = []
            subreddit_posts = reddit.subreddit(subred).hot(limit=10)
            for post in subreddit_posts:
                subreddit2submission[subred].append(post.id)
                parsed_post = parse_submission(post)
                for u in parsed_post["comments"]:
                    if u["author"] and u["author"] != current_user and u["author"] not in following_users:
                        following_users.append(u["author"])
                if parsed_post["submission"]["author"] not in following_users:
                    following_users.append(parsed_post["submission"]["author"])
                parsed_submissions[post.id] = parsed_post
                
        print(count)
        count += 1
                
    current_users = following_users 